# Q-learning

## Off-Policy 학습
* On-Policy: 타깃 정책과 행동 정책이 같은 경우(직접 경험)
* Off-Policy: 타깃 정책과 행동 정책이 다른 경우(간접 경험)
    - 과거의 경험을 재사용할 수 있다.
    - 사람의 데이터로부터 학습할 수 있다.
    - 일대다, 다대일 학습이 가능하다.
    
## Q러닝의 이론적 배경: 벨만 최적 방정식
최적 액션 밸류 $q_{*}(s,a)$는 이 세상에 존재하는 모든 정책들 중에 얻게 되는 가장 좋은 정책을 따를 때의 가치를 나타내는 함수이다.
<center>
    $q_{*}(s,a) = \underset{\pi}{max} q_{\pi}(s,a)$
</center>
$q_{*}$를 알게 되면 주어진 MDP에서 순간마다 최적의 행동을 취하면서 움직일 수 있다. 상태마다 $q_{*}$의 값이 가장 높은 액션을 취하면 되기 때문이다.
<center>
    $\pi_{*} = \underset{a}{argmax} q_{\pi}(s,a)$
</center>

그러나 최종적인 목표는 최적의 액션-가치 함수인 $q_{*}$를 찾는 것인데, 이는 벨만 최적 방정식을 기반으로 업데이트할 수 있다.
<center>
    벨만 최적 방정식: $q_{*}(s,a) = \mathbb{E}_{s^{\prime}}[r+\gamma \underset{a^{\prime}}{max}q_{*}(s^{\prime},a^{\prime})]$  
</center>
<center>
    Q러닝: $Q(S,A) \rightarrow Q(S,A) + \alpha (R + \gamma \underset{A^{\prime}}{max}Q(S^{\prime},A^{\prime})-Q(S,A))$
</center>


# Q-learning 구현

## 라이브러리 import 및 GridWorld 클래스

In [6]:
import random
import numpy as np

In [7]:
class GridWorld():
    def __init__(self):
        self.x = 0
        self.y = 0
    
    def step(self, a):
        # 0번 액션: 왼쪽, 1번 액션: 위, 2번 액션: 오른쪽, 3번 액션: 아래쪽
        if a == 0:
            self.move_left()
        elif a == 1:
            self.move_up()
        elif a == 2:
            self.move_right()
        elif a == 3:
            self.move_down()
        
        reward = -1 # 보상은 -1로 고정
        done = self.is_done()
        return (self.x, self.y), reward, done
    
    def move_right(self):
        if self.y == 1 and self.x in [0, 1, 2]:
            pass
        elif self.y == 3 and self.x in [2, 3, 4]:
            pass
        elif self.y == 6:
            pass
        else:
            self.y += 1
        
    def move_left(self):
        if self.y == 0:
            pass
        elif self.y == 3 and self.x in [0, 1, 2]:
            pass
        elif self.y == 5 and self.x in [2, 3, 4]:
            pass
        else:
            self.y -= 1
    
    def move_up(self):
        if self.x == 0:
            pass
        elif self.x == 3 and self.y == 2:
            pass
        else:
            self.x -= 1
            
    def move_down(self):
        if self.x == 4:
            pass
        elif self.x == 1 and self.y == 4:
            pass
        else:
            self.x += 1
    
    def is_done(self):
        if self.x == 4 and self.y == 6: # 목표 지점인 (4, 6)에 도달
            return True
        else:
            return False
    
    def reset(self):
        self.x = 0
        self.y = 0
        return (self.x, self.y)

In [8]:
class QAgent():
    def __init__(self):
        self.q_table = np.zeros((5, 7, 4)) # q밸류를 저장하는 변수. 모두 0으로 초기화
        self.eps = 0.9
    
    def select_action(self, s):
        # eps-greedy로 액션을 선택
        x, y = s
        coin = random.random()
        if coin < self.eps:
            action = random.randint(0, 3)
        else:
            action_val = self.q_table[x, y,:]
            action = np.argmax(action_val)
        return action
    
    def update_table(self, transition):
        s, a, r, s_prime = transition
        x, y = s
        next_x, next_y = s_prime
        # Q-learning 업데이트 식을 이용
        self.q_table[x, y, a] = self.q_table[x, y, a] + 0.1 * (r + np.amax(self.q_table[next_x, next_y,:] - self.q_table[x, y, a]))
            
    def anneal_eps(self):
        self.eps -= 0.01 # Q-learning에서는 epsilon이 좀 더 천천히 줄어들도록 함
        self.eps = max(self.eps, 0.2)
    
    def show_table(self):
        # 학습이 각 위치에서 어느 액션의 q 값이 가장 높았는지 보여줌
        q_list = self.q_table.tolist()
        data = np.zeros((5, 7))
        for row_idx in range(len(q_list)):
            row = q_list[row_idx]
            for col_idx in range(len(row)):
                col = row[col_idx]
                action = np.argmax(col)
                data[row_idx, col_idx] = action
        print(data)

In [9]:
def main():
    env = GridWorld()
    agent = QAgent()
    
    for n_epi in range(1000): # 총 1000 에피소드 동안 학습
        done = False
        
        s = env.reset()
        while not done:
            a = agent.select_action(s)
            s_prime, r, done = env.step(a)
            agent.update_table((s, a, r, s_prime))
            s = s_prime
        agent.anneal_eps()
        
    agent.show_table() # 학습이 끝난 결과를 출력

In [10]:
main()

[[3. 3. 0. 2. 2. 3. 3.]
 [3. 3. 0. 2. 2. 3. 3.]
 [3. 3. 0. 1. 0. 3. 3.]
 [2. 2. 2. 1. 0. 3. 3.]
 [0. 2. 2. 1. 0. 2. 0.]]
